## Gaussian filter from GCM filters
- Use a variable 30*R (R=Rossby radius) spatial filter
- Perform filtering on 0.25deg data as GCM filters can only work on gridded data. Most functions provided by Matthias Aengenheyster and much of this example is taken from https://github.com/eerie-project/EERIE_hackathon_2023/blob/main/RESULTS/eddy_composites-short.ipynb
- Intake catalog of EERIE data done by Fabian Wachsmann

Feb 2024, Matthias Aengenheyster (ECMWF) and Dian Putrasahan (MPIM)

In [1]:
# Some function from Matthias to be copied over. 
# cp -p /home/b/b382473/*.py /home/m/m300466/pyfuncs/MA
# cp -p /home/b/b382473/LR_filtered_1degree_r1440x721.nc /work/mh0256/m300466/ifsfesomgrids/RossbyRadius_filtered_1degree_r1440x721.nc
# cp -p /work/bm1344/a270228/EERIE_Hackathon/IFS-FESOM_CONTROL-1950/tco1279-NG5/1950/FESOM/025/daily/ssh_1950_19500101-19500131_025.nc /work/mh0256/m300466/ifsfesomgrids/ssh_1950_IFS25.nc

import gcm_filters
import numpy as np
import xarray as xr
import pandas as pd
import intake
import os,sys
from datetime import datetime
from dask.diagnostics import ProgressBar
import time as T

sys.path.insert(0, r'/home/m/m300466/pyfuncs/MA')
import geostats as gs
import filtering as fl

import warnings
warnings.filterwarnings("ignore")

In [2]:
# def ifs_to_latlon(ds):
#     '''
#     Uses the latitude-longitude information encoded in the regular grid IFS output to reconstructed the regular grid
#     '''
#     return ds.rename({'value':'latlon'}).set_index(latlon=("lat","lon")).unstack("latlon")

# def ifs_fix_time_for_monthly_data(ds):
#     '''
#     Monthly mean data has the time axis encoded incorrectly.
#     This function shifts it back by one day
#     '''
#     return ds.assign_coords(time=ds['time']- pd.Timedelta('1D'))

def get_area(da,mask=False):
    print('Computing grid-box area')
    import iris.analysis as ia
    if 'time' in da.dims:
        da = da.isel(time=0).drop('time')
    d = da.to_iris()
    d.coord('longitude').guess_bounds()
    d.coord('latitude').guess_bounds()

    area_weights = ia.cartography.area_weights(d)
    area = xr.ones_like(da) * area_weights
    if mask:
        area = area.where(~np.isnan(da))
    area = area.rename('area').load()
    area.attrs['long_name'] = 'grid_box_area'
    area.attrs['units'] = 'm^2'
    return area

def print_var(ds,filt=None):
    '''
    Print variables (varname,name) in Dataset. 
    If <filt> is provided, print only those where <filt> is present in the <name> attribute (ds[shortname].attrs['name'])

    Usage: print_var(ds,'wind')
    Output: printout of variables in ds whose name (long name in attributes, not short name to access) contains 'wind', e.g.
        10si  :   10 metre wind speed
        10u  :   10 metre U wind component
        10v  :   10 metre V wind component
    '''
    if filt:
        [print('%10s  :   %s' % (d,ds[d].attrs['name'])) for d in ds if filt.lower() in ds[d].attrs['name'].lower()]
    else:
        [print('%10s  :   %s' % (d,ds[d].attrs['name'])) for d in ds]




## Open and inspect catalogue


In [3]:
cat = intake.open_catalog("https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/eerie.yaml")
model = 'ifs-fesom2-sr'
# model = 'icon-esm-er'
expid = 'eerie-control-1950'
gridspec = 'gr025'
realm='ocean'
# realm='atmos'
cat_regrid = cat['dkrz.disk.model-output'][model][expid][realm][gridspec]
print(list(cat_regrid))

['daily', 'monthly']


In [4]:
ds1 = cat_regrid['daily'].to_dask()
# ds1 = cat_regrid['daily_2d'].to_dask()
# ds1 = cat_regrid['2d_daily_mean'].to_dask()

timesel = dict(time=slice('1950-01-01','1950-12-31'))
ds_subset1 = ds1.sel(timesel)
# ds_subset1 = ds1
datearr = np.array([pd.Timestamp(t).to_pydatetime() for t in ds_subset1.time.values])


In [ ]:
varname='sst'
print('High pass filter daily '+varname+' for year='+str(datearr[0].year)+'-'+str(datearr[-1].year))
wavelength='30R'
bnds = dict(lat=slice(-85,85))

# Directories
scratch = '/scratch/m/m300466/'
datadir = scratch+expid+'/'+gridspec+'/'
if not os.path.exists(datadir+'/'+model):
    os.makedirs(datadir+'/'+model)
    
vardir=datadir+model+'/'+varname
filtdir=vardir+'/Gaussian'
smdatadir=filtdir+'/sm'+wavelength
if not os.path.exists(vardir):
        os.makedirs(vardir)
if not os.path.exists(filtdir):
        os.makedirs(filtdir)
if not os.path.exists(smdatadir):
        os.makedirs(smdatadir)
        


In [ ]:
def get_da_field(ds, varname, bnds):
    ds = ds[[varname]].sel(bnds)
    # ds = fl.define_grid_metrics(ds)
    
    # wetmask_gl = xr.where(np.isnan(ds[varname].isel(time=0)),0,1).load()
    wetmask_gl = xr.where(np.isnan(ds[varname]),0,1).load()
    # Set mask to zero at southern (irrelevant - Antarctica) and northern (Arctic) boundary to prevent "outflow" of information
    wetmask_gl[0,:] = 0
    wetmask_gl[-1,:] = 0
    wet_mask = wetmask_gl.drop('time').chunk({'lat': len(ds.lat), 'lon': len(ds.lon)})  # 1 chunk

    da_field = ds[varname].where(wet_mask)
    da_field = da_field.chunk({'lat': len(ds.lat), 'lon': len(ds.lon)})  # 1 chunk
    return da_field.to_dataset()
    
def setup_filter(ds, varname, bnds):
    '''
    Setup filter based on <varname> in ds, limited to geographical bnds
    Using a Gaussian filter with varying spatial scales (factor * Rossby radius)
    '''
    ds = ds[[varname]].sel(bnds)
    ds = fl.define_grid_metrics(ds)

    dxw = ds.dxc.rename({'lon_g':'lon'}).assign_coords(lon=ds.lon).rename('dxw')#.sel(lat=slice(-60,60))
    dyw = ds.dyg.rename({'lon_g':'lon'}).assign_coords(lon=ds.lon).rename('dyw')#.sel(lat=slice(-60,60))
    dxs = ds.dxg.rename({'lat_g':'lat'}).assign_coords(lat=ds.lat).rename('dxs')#.sel(lat=slice(-60,60))
    dys = ds.dyc.rename({'lat_g':'lat'}).assign_coords(lat=ds.lat).rename('dys')#.sel(lat=slice(-60,60))
    # grid spacings, between cell centers (_c) and cell boundaries (_g), based on the MITgcm terminology
    # dxw = x-spacing centered at western cell edge
    # dyw = y-spacing centered at western cell edge
    # dxs = x-spacing centered at southern cell edge
    # dys = y-spacing centered at southern cell edge

    dx_min = min(
        dxw.min(),
        dyw.min(),
        dxs.min(),
        dys.min()).values
    
    area = gs.get_area(ds[varname])

    wetmask_gl = xr.where(np.isnan(ds[varname].isel(time=0)),0,1).load()
    # wetmask_gl = xr.where(np.isnan(ds[varname]),0,1).load()
    # Set mask to zero at southern (irrelevant - Antarctica) and northern (Arctic) boundary to prevent "outflow" of information
    wetmask_gl[0,:] = 0
    wetmask_gl[-1,:] = 0
    
    wet_mask = wetmask_gl.drop('time').chunk({'lat': len(ds.lat), 'lon': len(ds.lon)})  # 1 chunk
    area = area.chunk({'lat': len(ds.lat), 'lon': len(ds.lon)})  # 1 chunk
    dxw = dxw.chunk({'lat': len(ds.lat), 'lon': len(ds.lon)}) # 1 chunk
    dyw = dyw.chunk({'lat': len(ds.lat), 'lon': len(ds.lon)}) # 1 chunk
    dxs = dxs.chunk({'lat': len(ds.lat), 'lon': len(ds.lon)}) # 1 chunk
    dys = dys.chunk({'lat': len(ds.lat), 'lon': len(ds.lon)}) # 1 chunk

    # da_field = ds[varname].where(wet_mask)
    # da_field = da_field.chunk({'lat': len(ds.lat), 'lon': len(ds.lon)})  # 1 chunk

    # https://gcm-filters.readthedocs.io/en/latest/examples/example_filter_types.html#preparing-the-grid-input-variables  
    # ======================
    # ROSSBY-RADIUS BASED FILTERING
    # ======================
    LR_gl = xr.open_dataarray('/home/b/b382473/LR_filtered_1degree_r1440x721.nc').sel(bnds)
    #is the Rossby radius regridded to the 1/4 degree grid from PHC3.0 climatology
    
    # factor 30, (300-1500 km)
    LRfactor = 30
    LRmax = 1500

    # small: factor 5 (50-250 km)
    # LRfactor = 5
    # LRmax = 250
    # large: factor 90 (900-4500 km)
    # LRfactor = 90
    # LRmax = 4500
    # medium: factor 12 (120-600 km)
    # LRfactor = 12
    # LRmax = 600

    LRmin = 10 # set minimum length to 10
    LR_min_max30_gl = xr.where(LR_gl<LRmin,LRmin,LR_gl)
    LR_min_max30_gl = xr.where(LR_min_max30_gl*LRfactor>LRmax,LRmax,LR_min_max30_gl*LRfactor) # 30 x rossby radius, capped at 1500 km
    #LRfactor, LRmax, LRmin: multiple of Rossby radius, and maximum/minimum cutoffs
    #NB: minimum is done before the  factor, but the maximum is done after
    # Largest range of LR_min_max30_gl = (LRfactor * LRmin , LRmax) 

    # L_max = LR.max().values
    L_max_gl = LR_min_max30_gl.max().values
    filter_scale = L_max_gl
    kappa_min_max30_gl = LR_min_max30_gl**2 / L_max_gl**2
    kappa_min_max30_gl = kappa_min_max30_gl.chunk({'lat': len(ds.lat), 'lon': len(ds.lon)}) # 1 chunk

    kappa_w = kappa_min_max30_gl.copy()
    kappa_s = kappa_min_max30_gl.copy()
    # kappa_w and kappa_s are the diffusivity in x and y, that needs a lengthscale for scaling.
    # For a constant lengthscale, they are both =1, and one can change them separately to have different filter lengths per direction
    # We should keep kappa to between 0 and 1. 
    
    
    specs = dict(
        filter_scale=filter_scale * 1e3, #from km to m
        dx_min=dx_min,
        filter_shape=gcm_filters.FilterShape.GAUSSIAN,
        grid_type=gcm_filters.GridType.IRREGULAR_WITH_LAND,
    )

    filter_cpu = gcm_filters.Filter(grid_vars={
        'area': area,
        'wet_mask': wet_mask,
        'dxw': dxw,
        'dyw': dyw,
        'dxs': dxs,
        'dys': dys,
        'kappa_w': kappa_w,
        'kappa_s': kappa_s,
        }, **specs)
    print('Filter: ',filter_cpu)
    
    return filter_cpu

def filter_data(filter_obj, da_field, load=False):
    fields_filtered = filter_obj.apply(
        da_field, dims=['lat', 'lon'])
        # ds, dims=['lon'])
    if load:
        with ProgressBar():
            fields_filtered.load()
    return fields_filtered

def gaussian_gcm_filter(ds_subset1,varname,bnds,datearr,tt,smdatadir,wavelength,filter_cpu): 
    date = datearr[tt]
    # Load data from xarray into netcdf4 type
    ds_subset = ds_subset1[[varname]].sel(bnds).isel(time=tt)
    # da_ssh.time.encoding.pop("_FillValue",None)
    da_field=get_da_field(ds_subset, varname, bnds)

    print('High pass filter of '+varname+' for '+date.strftime('%Y%m%d'))
    fields_filtered=filter_data(filter_cpu, da_field, load=False)

    if varname=='to' or varname=='so' or varname=='rho':
        zidx=1
        foutname=smdatadir+'/'+varname+'_'+str(zidx)+'_'+date.strftime('%Y%m%d')+'_sm'+wavelength+'.nc'
    else:
        foutname=smdatadir+'/'+varname+'_'+date.strftime('%Y%m%d')+'_sm'+wavelength+'.nc'

    fields_filtered.to_netcdf(path=foutname)



    

In [ ]:
# filter_cpu=setup_filter(ds_subset1, varname, bnds)
# dsifs=cat['dkrz.disk.model-output']['ifs-fesom2-sr']['eerie-control-1950']['ocean']['gr025']['daily'].to_dask()
# filter_cpu=setup_filter(dsifs.sel(time=slice('1950-01-01','1950-01-03')),'ssh',dict(lat=slice(-85,85)))

dssshifs=xr.open_dataset('/work/mh0256/m300466/ifsfesomgrids/ssh_1950_IFS25.nc')
filter_cpu2=setup_filter(dssshifs.sel(time=slice('1950-01-01','1950-01-03')),'ssh',dict(lat=slice(-85,85)))


In [ ]:
for tt in range(len(datearr)):
    date=datearr[tt]
    gaussian_gcm_filter(ds_subset1,varname,bnds,datearr,tt,smdatadir,wavelength,filter_cpu2)